In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.sequence import pad_sequences




In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Download the Indonesian stopwords if not already downloaded
nltk.download('stopwords')

# Load the mapping between informal and formal words
df_new_kamusalay = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
df_new_kamusalay = df_new_kamusalay.rename(columns={0: 'original', 1: 'new'})
df_new_kamusalay = df_new_kamusalay.set_index('original')['new'].to_dict()

# Load the list of abusive words
df_abusive = pd.read_csv('abusive.csv', delimiter='\t', header=None)
df_abusive.columns = ['ABUSIVE']

# Define the Indonesian stopwords
stop_words = set(stopwords.words('indonesian'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data train

data_train = pd.read_csv('train_preprocess.csv', sep='\t',header = None, encoding='latin1')
data_test = pd.read_csv('data.csv', sep='\t',header = None, encoding='latin1')

df = data_train.append(data_test, ignore_index=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_4284\2024862704.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = data_train.append(data_test, ignore_index=True)


In [4]:
df.head()

,0,1
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [5]:
df = df.rename(columns={0: 'Text', 1: 'Sentimen'})
df.head()

,Text,Sentimen
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


# CLeansing Data

In [6]:
# Clean the text by removing stopwords
def cleansing_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word.lower() not in stop_words)
    return text.lower()

# Clean the text using regex
def cleansing_byusingregex(text):
    text = re.sub('-', ' ', text) # Remove tanda strip
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) # Remove all symbols
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = re.sub('\n', ' ', text) # Remove every '\n'
    text = re.sub('#', ' ', text) # Remove every retweet symbol
    text = re.sub('user', ' ', text) # Remove every username
    text = re.sub('USER', ' ', text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', ' ', text) # Remove every URL
    text = re.sub('[\d\.]+', '', text) # Remove numbers
    text = re.sub('!', ' ', text)
    text = re.sub(r'[\t\s]+', ' ', text) # Menghapus spasi berlebih
    text = re.sub(r'[^\w\s]', '', text) # Menghapus tanda baca
    return text.lower()

# Change words using the mapping and remove abusive words
def change_word(text):
    # Cleansing Kata Alay
    text = ' '.join([df_new_kamusalay[word] if word in df_new_kamusalay else word for word in text.split(' ')])
    # Cleansing Kata Abusive
    text = ' '.join(['***' if word in df_abusive.ABUSIVE.values else word for word in text.split(' ')])
    text = text.strip() # Remove leading and trailing spaces
    return text.lower()

# Preprocessing function
def preprocessing(text):
    text = cleansing_stopwords(text) # Remove stopwords
    text = cleansing_byusingregex(text)
    text = change_word(text) # Change words and remove abusive words
    return text

def pred_sentiment(text):
    cleaned_text = preprocessing(text) # Preprocess the input text
    text_sequence = tokenizer.transform([cleaned_text]) # Transform the preprocessed text
    padded_sequence = pad_sequences(text_sequence.toarray()) # Convert to dense matrix and pad the transformed sequence
    predictions = loaded_model.predict(padded_sequence, batch_size=10)[0] # Make predictions
    return predictions

def pred(predictions):
    if predictions.argmax() == 0:
        print('negative')
    elif predictions.argmax() == 1:
        print('neutral')
    elif predictions.argmax() == 2:
        print('positive')


In [7]:
# Load the pre-trained model
loaded_model = load_model('lstm4.h5')

# Inisialisasi tokenizer
tokenizer = TfidfVectorizer()

# Load data from CSV file
data = pd.read_csv('data.csv', encoding='latin1')

# Extract the text data from the CSV file
data_text = data['Tweet'].tolist()

# Fitting tokenizer
tokenizer.fit(data_text)


TfidfVectorizer()

In [8]:
Tweet = "- Berusaha terus agar pintar ! ! !',1,1,1,0,0,0,0,0,1,1,0,0"
Sentimen = pred_sentiment(Tweet)

print('=============================')
print('         Sentimen Analisis')
print('=============================')
print('Tweet: {}'.format(Tweet))
print('Tweet Preprocessing: {}'.format(preprocessing(Tweet)))
print('-----------------------------')
print('Sentimen :' )
pred(Sentimen)
print('=============================')

1/1 [==============================] - 3s 3s/step
         Sentimen Analisis
Tweet: - Berusaha terus agar pintar ! ! !',1,1,1,0,0,0,0,0,1,1,0,0
Tweet Preprocessing: berusaha pintar
-----------------------------
Sentimen :
negative


# Predict menggunakan Package yang di buat

In [11]:
from preprocessingtextpackage.normalisasitext import text_preprocessing 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

# Load the pre-trained model
loaded_model = load_model('lstm4.h5')

# Inisialisasi tokenizer
tokenizer = TfidfVectorizer()

# Load data from CSV file
data = pd.read_csv('data.csv', encoding='latin1')

# Extract the text data from the CSV file
data_text = data['Tweet'].tolist()

# Fitting tokenizer
tokenizer.fit(data_text)

text = "- Berusaha terus agar pintar ! ! !',1,1,1,0,0,0,0,0,1,1,0,0."
preprocessed_text = text_preprocessing(text)
Sentimen = pred_sentiment(text)

print('        _ Model LSTM _       ')
print('       Sentimen Analisis     ')
print('\n')
print('Tweet: {}'.format(text))
print('Tweet Preprocessing: {}'.format(preprocessed_text))
print('Sentimen :' )
pred(Sentimen)
print('=============================')


1/1 [==============================] - 2s 2s/step
        _ Model LSTM _       
       Sentimen Analisis     


Tweet: - Berusaha terus agar pintar ! ! !',1,1,1,0,0,0,0,0,1,1,0,0.
Tweet Preprocessing: berusaha pintar
Sentimen :
negative
